# Introduction to Quantitative Finance

Copyright (c) 2019 Python Charmers Pty Ltd, Australia, <https://pythoncharmers.com>. All rights reserved.

<img src="img/python_charmers_logo.png" width="300" alt="Python Charmers Logo">

Published under the Creative Commons Attribution-NonCommercial 4.0 International (CC BY-NC 4.0) license. See `LICENSE.md` for details.

Sponsored by Tibra Global Services, <https://tibra.com>

<img src="img/tibra_logo.png" width="300" alt="Tibra Logo">


## Module 1.1: Distributions and Random Processes

### 1.1.3: Moments

Moments describe distributions. We'll focus on the normal (and normal-ish) distributions for now, but will look at other distributions later.

<text>
矩描述了分布。我们目前将专注于正态（或近似正态）分布，但稍后会研究其他分布。
</text>

A normal distribution is fully described by the first two moments, which are the mean and the variance. Reviewing the help for the `stats.norm` function, we can see these are the only two parameters we can input (see the docstring of the function).

In [21]:
# 运行初始化脚本setup.ipy
%run setup.ipy

<div class="alert alert-success">
    Note: it's worth opening up setup.ipy and seeing what's in there. This file will be run at the start of most of our notebooks.
</div>

In [22]:
# 查看 scipy.stats.norm 函数的帮助文档

stats.norm?

Signature:       stats.norm(*args, **kwds)
Type:            norm_gen
String form:     <scipy.stats._continuous_distns.norm_gen object at 0x7f47f6bc4e60>
File:            ~/miniconda3/envs/quant_finance/lib/python3.12/site-packages/scipy/stats/_continuous_distns.py
Docstring:      
A normal continuous random variable.

The location (``loc``) keyword specifies the mean.
The scale (``scale``) keyword specifies the standard deviation.

As an instance of the `rv_continuous` class, `norm` object inherits from it
a collection of generic methods (see below for the full list),
and completes them with details specific for this particular distribution.

Methods
-------
rvs(loc=0, scale=1, size=1, random_state=None)
    Random variates.
pdf(x, loc=0, scale=1)
    Probability density function.
logpdf(x, loc=0, scale=1)
    Log of the probability density function.
cdf(x, loc=0, scale=1)
    Cumulative distribution function.
logcdf(x, loc=0, scale=1)
    Log of the cumulative distribution function.
s

As noted in that description, the first moment, the mean, is referred to as the location. It specifies where the normal distribution is centred.

<text>
正如该描述中所指出的，第一矩，即均值，被称为位置。它指定了正态分布的中心位置。
</text>


In [58]:
# 定义一个函数，用于绘制正态分布的直方图
def plot_histogram_normal(mean, standard_deviation, color):
    # 创建一个正态分布对象，使用指定的均值和标准差
    distribution = stats.norm(mean, standard_deviation)
    # 从该分布中随机生成10000个样本，并转换为DataFrame格式
    normal_values = pd.DataFrame({"value": distribution.rvs(10000)})

    # 使用Altair创建直方图
    chart = alt.Chart(normal_values).mark_bar().encode(
        # X轴设置为数值，并将数据分成最多100个箱
        alt.X("value", bin=alt.Bin(maxbins=100)),
        # Y轴显示每个箱的计数
        y='count()',
        # 设置直方图的颜色
        color=alt.value(color)
    )
    return chart

# 创建两个不同参数的正态分布直方图
# 第一个直方图：均值为0，标准差为1，红色
chart_1 = plot_histogram_normal(0, 1, "red")
# 第二个直方图：均值为3，标准差为1，蓝色
chart_2 = plot_histogram_normal(3, 5, "blue")
# 将两个直方图叠加显示
chart_1 + chart_2

alt.LayerChart(...)

The mean is the expected value of the distribution. Given all other things equal, if we chose *n* values randomly from this distribution, the average value (mean) would be equal to the mean of the distribution. This might seem like circular knowledge, but note the values are computed in different ways:

<text>
均值是分布的期望值。在其他条件相同的情况下，如果我们从该分布中随机选择 *n* 个值，平均值（均值）将等于分布的均值。这看起来像是循环知识，但请注意这些值是以不同的方式计算的：
</text>


In [24]:
# 设置实际均值为57
actual_mean = 57
# 设置标准差为0到10之间的随机数
standard_deviation = random.random() * 10
# 设置试验次数为100000
N_TRIALS = 100000

# 创建一个均值为actual_mean，标准差为standard_deviation的正态分布
distribution = stats.norm(actual_mean, standard_deviation)
# 从该分布中随机采样N_TRIALS个值
normal_values = distribution.rvs(N_TRIALS)

In [25]:
# 计算样本均值与实际均值之间的误差
error = np.mean(normal_values) - actual_mean
# 打印实际均值和计算得到的样本均值
print("The actual mean was {actual_mean}, while the computed mean was {computed_mean:.3f}".format(
    actual_mean=actual_mean, computed_mean=np.mean(normal_values)))
# 打印误差值
print("This gives an error of {error:.3f}".format(error=error))

The actual mean was 57, while the computed mean was 57.016
This gives an error of 0.016


Note that the mean is not the median, although in a normal distribution, they are usually about the same (and theoretically they are the same value). The median is not a "moment".

In [26]:
# 计算从正态分布中抽样的数据的中位数

The second moment of a normal distribution is the variance, also known as the scale factor of the distribution. It is the expected value of the squared difference between a random value and the mean:

<text>
正态分布的二阶矩是方差，也称为分布的尺度因子。它是随机值与均值之间平方差的期望值：
</text>

$V=\frac{1}{n}\sum^n_{i=0}(X_i-\mu)^2$

Note that the square in the result makes the unit squared as well. For instance, if our measurements were in metres $m$, the variance would be in metres squared, $m^2$. As a result, it's not directly comparable to the initial value. For instance:

In [27]:
# 计算正态分布样本的方差
V = np.var(normal_values)
# 打印方差值
V

79.28029656705743

We can not directly compare this to our original units, i.e. we can not say the variance is "about 0.5% of the mean".
Such a statment is meaningless as the units are different. 
For that reason, we usually use the square root of the variance, known as the standard deviation, which is in the same units as X, and is therefore comparable in such a way:

<text>
出于这个原因，我们通常使用方差的平方根，即标准差，它与 X 的单位相同，因此可以以这种方式进行比较：
</text>

$V=\sigma^2=\frac{1}{n}\sum^n_{i=0}(X_i-\mu)^2$

It is this "standard deviation" that is the second input into our `stats.norm` function:

In [28]:
# 创建两个正态分布直方图并叠加显示
chart_3 = plot_histogram_normal(0, 1, "green")  # 创建均值为0、标准差为1的绿色正态分布直方图
chart_4 = plot_histogram_normal(6, 2, "orange")  # 创建均值为6、标准差为2的橙色正态分布直方图
chart_3 + chart_4  # 将两个直方图叠加显示在同一个图表中

alt.LayerChart(...)

The larger standard deviation makes the distribution more spread out, but it is the same shape, simply "scaled".

### Further Moments

There are two further moments in common use. The third sequentially is called the skew.
It can be visualised as "pulling" the distribution to the left (negative skew) or right (positive skew).

A normal distribution is symmetrical, and has a skew of 0. This is why it does not appear in the equation or function calls to generate the normal distribution.

The fourth standardised moment is the kurtosis, more commonly seen in financial data than in many other datasets. A higher value indicates "fatter tails" than a standard normal distribution. The kurtosis value of a normal distribution is always 3 - we consider this our baseline when interpreting the kurtosis value of other distributions.

In [79]:
# 使用skewnorm函数创建一个偏度为4的偏态正态分布对象

stats.skewnorm(4)

In [80]:
# 查看 scipy.stats 中 skewnorm 函数的帮助文档，该函数用于生成偏态正态分布

stats.skewnorm?

Signature:       stats.skewnorm(*args, **kwds)
Type:            skewnorm_gen
String form:     <scipy.stats._continuous_distns.skewnorm_gen object at 0x7f47f4ff9430>
File:            ~/miniconda3/envs/quant_finance/lib/python3.12/site-packages/scipy/stats/_continuous_distns.py
Docstring:      
A skew-normal random variable.

As an instance of the `rv_continuous` class, `skewnorm` object inherits from it
a collection of generic methods (see below for the full list),
and completes them with details specific for this particular distribution.

Methods
-------
rvs(a, loc=0, scale=1, size=1, random_state=None)
    Random variates.
pdf(x, a, loc=0, scale=1)
    Probability density function.
logpdf(x, a, loc=0, scale=1)
    Log of the probability density function.
cdf(x, a, loc=0, scale=1)
    Cumulative distribution function.
logcdf(x, a, loc=0, scale=1)
    Log of the cumulative distribution function.
sf(x, a, loc=0, scale=1)
    Survival function  (also defined as ``1 - cdf``, but `sf` is so

In [81]:
# 定义一个函数，用于绘制偏态正态分布的直方图
def plot_histogram_normal_skewed(mean, standard_deviation, skew, color):
    # 创建偏态正态分布对象，设置偏度、位置(均值)和尺度(标准差)参数
    distribution = stats.skewnorm(skew, loc=mean, scale=standard_deviation)
    # 从分布中随机生成10000个样本，并转换为DataFrame格式
    normal_values = pd.DataFrame({"value": distribution.rvs(10000)})

    # 使用Altair创建直方图
    chart = alt.Chart(normal_values).mark_bar().encode(
        # X轴设置为数值，并将数据分成最多100个箱
        alt.X("value", bin=alt.Bin(maxbins=100)),
        # Y轴显示每个箱中的计数
        y='count()',
        # 设置直方图的颜色
        color=alt.value(color)
    )
    return chart

In [102]:
# 调用plot_histogram_normal_skewed函数绘制偏态正态分布直方图
# 参数说明:
# - 均值(mean) = 0
# - 标准差(standard_deviation) = 1  
# - 偏度(skew) = 2，表示正偏(向右偏)
# - 颜色 = 蓝色
plot_histogram_normal_skewed(0, 1, 2, "blue")

alt.Chart(...)

For seeing the kurtosis in action, let's look at some data. We will load in the AAPL stock price from a h5 file:

<text>
为了观察峰度的实际效果，让我们来看一些数据。我们将从 h5 文件中加载 AAPL 股票价格：
</text>


In [83]:
# 从pickle文件中读取苹果公司(AAPL)的股票数据
aapl = pd.read_pickle("data/AAPL.pkl")

In [84]:
# 显示苹果公司股票数据的前5行记录
aapl.tail()

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2000-01-03,104.87,112.50,101.69,111.94,19144400,27.74
2000-01-04,108.25,110.62,101.19,102.50,18310000,25.40
2000-01-05,103.75,110.56,103.00,104.00,27818000,25.77
2000-01-06,106.12,107.00,95.00,95.00,27443200,23.54
2000-01-07,96.50,101.00,95.50,99.50,16463200,24.66


#### Exercises

1. Compute the increase in price for each day (Close - Open)
2. Plot a histogram of these increases
3. Investigate the `stats.skew` and `stats.kurtosis` functions to compute the third and fourth moment of the dataset.

*For solutions, see `solutions/moments.py`*

#### Extended exercise

Quandl has a python module for extracting datasets. The documentation is available at https://www.quandl.com/tools/python

Install this module, and review the documentation to obtain stock prices for the following four tech giants:
* IBM
* Google
* Apple (more up-to-date than our dataset)
* Amazon

Compute the skew and kurtosis of each stock, and compare the results. Looking at the histograms of the stock prices, the skew and the kurtosis, what does this tell you about the usefulness of these moments?

Note: Extended exercises are more open-ended than normal exercises, and may take significantly longer to complete. They also tend to be harder than other exercises. 

In [108]:
# Exercise 1
aapl['Gain'] = aapl['Close'] - aapl['Open']

# 增强版直方图
chart = alt.Chart(aapl).mark_bar(
    opacity=0.5,  # 设置透明度
    color='blue'  # 设置颜色
).encode(
    alt.X("Gain", 
          bin=alt.Bin(maxbins=50),
          title="价格变化（美元）"  # X轴标题
    ),
    alt.Y('count()',
          title="频率"  # Y轴标题
    )
).properties(
    title="每日价格变化分布",  # 图表标题
    width=600,  # 图表宽度
    height=400  # 图表高度
)

chart.display()

# Exercise 3
skew = stats.skew(aapl['Gain'])
print("The skew is {}".format(skew))
kurtosis = stats.kurtosis(aapl['Gain'])
print("The kurtosis is {}".format(kurtosis))

alt.Chart(...)

The skew is -0.3589561655618472
The kurtosis is 12.862787458978701


In [85]:
# 计算每日价格变化
aapl['Price_Change'] = aapl['Close'] - aapl['Open']
aapl.head()

# 创建价格变化的直方图
# 使用 Altair 绘制红色柱状图
# bin=alt.Bin(maxbins=100) 将数据分成最多100个区间
chart = alt.Chart(aapl).mark_bar().encode(
        alt.X("Price_Change", bin=alt.Bin(maxbins=100)),
        y='count()',
        color=alt.value('red'))
chart.display()

# 打印价格变化的偏度(skewness)和峰度(kurtosis)
# 偏度衡量分布的对称性，峰度衡量分布尾部的厚度
print("Skew: " + str(stats.skew(aapl['Price_Change'])))
print("Kurtosis: " + str(stats.kurtosis(aapl['Price_Change'])))

alt.Chart(...)

Skew: -0.3589561655618472
Kurtosis: 12.862787458978701


*For solutions, see `solutions/moments.py`*

### Z-scores

A "z-score" is a common normalisation method used for data. It removes the scale of the data, and instead considers the size of the data in terms of the standard deviation. It is a transformation of the data from one scale to another, using the mean and standard deviation:

In [86]:
# 创建一个包含7个数值的numpy数组
original_data = np.array([10, 20, 5, 105, 30, 17, 19], dtype=np.float32)
# 计算数组的平均值
m = np.mean(original_data)
# 计算数组的标准差
s = np.std(original_data)

The transformation is to subtract the mean, and divide by the standard deviation:

In [87]:
# 计算Z分数：将原始数据标准化，通过减去均值并除以标准差，得到每个数据点偏离平均值的标准差倍数
zscores = (original_data - m) / s

In [88]:
# 打印标准化后的数据(z-scores)，即原始数据减去均值后除以标准差，用于衡量每个数据点偏离平均值的程度
zscores

array([-0.612737  , -0.29735765, -0.77042663,  2.3833666 ,  0.01802167,
       -0.39197144, -0.3288956 ], dtype=float32)

The values of the z-scores are normalised, allowing us to compare data from different scales - for instance, comparing the stock prices between AAPL and MSFT for a period of one month, where direct comparisons are initially hard. 

<text>
z-score 的值经过标准化处理，使得我们能够比较不同尺度的数据——例如，比较 AAPL 和 MSFT 在一个月内的股票价格，而直接比较最初是困难的。
</text>

Let's load some data from Quandl. To do that, create a file called `my_secrets.py` and create a value called `QUANDL_API_KEY` and set that equal to your API key from Quandl. You can obtain one by signing up at https://www.quandl.com/tools/api and then viewing your profile page at https://www.quandl.com/account/profile

You can copy the file `my_secrets_template.py` to create this file for you. Just copy the file and fill out the data. Ensure this file is in the same directory as your notebooks.

In [89]:
%%writefile my_secrets.py

# 存储 Quandl API 密钥的变量
QUANDL_API_KEY = "ue4SAPctpsjD3UJYZ2o1"

Overwriting my_secrets.py


In [90]:
# 导入 Quandl 库，用于获取金融数据
import quandl
# 导入包含 API 密钥的本地配置文件
import my_secrets
# 设置 Quandl API 密钥，用于认证访问 Quandl 数据服务
quandl.ApiConfig.api_key = my_secrets.QUANDL_API_KEY

In [91]:
# 从Quandl获取MSFT和AAPL的股票数据
data = quandl.get_table('WIKI/PRICES', ticker = ['MSFT', 'AAPL'], # 指定获取微软和苹果的股票数据
                        qopts = { 'columns': ['ticker', 'date', 'adj_close'] }, # 只选择股票代码、日期和经调整收盘价这三列
                        date = { 'gte': '2017-01-01', 'lte': '2024-01-01' }, # 设置日期范围从2017年到2019年
                        paginate=True) # 启用分页获取数据，用于处理大量数据

In [92]:
# 随机抽样显示数据中的5行
data.sample(5)
# 查看数据对象的类型
type(data)

pandas.core.frame.DataFrame

If we compare the means, we see that AAPL has a higher adjusted close value.

<text>
如果我们比较均值，我们会发现 AAPL 的调整后收盘价更高。
</text>


In [93]:
# 按股票代码分组计算每支股票的平均调整收盘价

data.groupby("ticker")['adj_close'].mean()

ticker
AAPL    154.137248
MSFT     75.098922
Name: adj_close, dtype: float64

However, we might be more interested to see whether movements swing wildly, or are stable with regard to the current price.

In [94]:
# 使用Altair创建股票收盘价分布的柱状图
alt.Chart(data).mark_bar(opacity=0.4).encode(
    # X轴：设置为调整后的收盘价(adj_close)，并将数据分成最多30个区间
    x=alt.X("adj_close", bin=alt.Bin(maxbins=30)),
    # Y轴：统计每个区间的数量，stack=None表示不堆叠不同股票的数据
    y=alt.Y('count()', stack=None),
    # column='ticker',  # 注释掉的代码：原本用于按股票代码分列显示
    # 使用不同颜色区分不同的股票代码
    color='ticker',
)

alt.Chart(...)

To truly compare these distributions, we need to convert them to z-scores first, which gives us more information about the relative stock price movements:

<text>
要真正比较这些分布，我们首先需要将它们转换为 z 分数，这为我们提供了有关股票价格相对变动的更多信息：
</text>


In [95]:
# 显示数据框的所有列名

data.columns

Index(['ticker', 'date', 'adj_close'], dtype='object')

In [96]:
# 将数据透视为以日期为索引、股票代码为列的价格矩阵
prices = data.pivot(columns="ticker", index="date", values='adj_close')
# 计算每支股票的Z分数：(价格-均价)/标准差
z_scores = (prices - prices.mean())/prices.std()
# 显示前5行Z分数数据
z_scores.head()

ticker,AAPL,MSFT
date,,
2017-01-03,-2.399152,-1.330589
2017-01-04,-2.406966,-1.356847
2017-01-05,-2.371503,-1.356847
2017-01-06,-2.293364,-1.306206
2017-01-09,-2.228449,-1.324962


In [97]:
# 创建一个柱状图来展示标准化后的股票收盘价分布
alt.Chart(z_scores.melt(value_name="z_score_adj_close")).mark_bar(opacity=0.4).encode(
    # X轴：标准化后的收盘价，分成最多30个区间
    x=alt.X("z_score_adj_close", bin=alt.Bin(maxbins=30)),
    # Y轴：每个区间的计数，不堆叠显示
    y=alt.Y('count()', stack=None),
    # column='ticker',  # 注释掉的列分组
    # 使用不同颜色区分不同的股票代码
    color='ticker',
)

alt.Chart(...)

We can now compare the distributions, visually and directly against each other. This specific analysis doesn't tell us much, but we can use z-scores to compare distributions of data from different scales, as we saw above.

<text>
我们现在可以直观地直接比较这些分布。这种特定的分析并没有告诉我们太多信息，但正如我们上面所看到的，我们可以使用 z 分数来比较来自不同尺度的数据分布。
</text>


#### Exercise

Perform the same analysis, but using the increase in adjusted closing price in a given day, rather than the absolute value.

In [98]:
# 计算每日收益率
returns = prices.pct_change().iloc[1:,:]
# 对收益率进行标准化处理，计算z分数
z_rets = (returns - returns.mean())/returns.std()

# 创建柱状图可视化标准化后的收益率分布
# 使用altair绘制直方图，设置透明度为0.4
alt.Chart(z_rets.melt(value_name="z_score_returns")).mark_bar(opacity=0.4).encode(
    # X轴显示标准化收益率，最多分30个箱
    x=alt.X("z_score_returns", bin=alt.Bin(maxbins=30)),
    # Y轴显示每个箱的计数，不堆叠
    y=alt.Y('count()', stack=None),
    # column='ticker',  # 注释掉的列分组
    # 根据股票代码设置不同颜色
    color='ticker',
)

/tmp/ipykernel_135996/4210873990.py:2: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().iloc[1:,:]


alt.Chart(...)

*For solutions, see `solutions/adjusted_increases.py`*